In [ ]:
# feed in features of embeddings
#create sparse matrix from item-feature matrix
#calc cosine sim from matrix
#Should be able to select different typs of sim metrics


In [2]:

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [45]:
df = pd.DataFrame([np.random.rand(1, 20)[0], np.random.rand(1, 20)[0]]).T
df.columns = ["f1", "f2"]
df.head()

,f1,f2
0,0.853281,0.331458
1,0.910494,0.836155
2,0.237426,0.687891
3,0.879508,0.892442
4,0.956701,0.792583


In [61]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


class ContentRecommender(object):

    def get_similar_recommendations(self, seed_item, feature_matrix, similarity_metric, n):
        ''' Return top n similar items to a seed item '''
        
        if similarity_metric not in ["cosine", "euclidean", "manhattan", "jaccard"]:
            return ValueError("similarity_metric must be cosine, euclidean, manhattan, or jaccard")

        #get style variant's feature vector
        item_vector = np.array(feature_matrix.loc[seed_item]).reshape(1, -1)

        #calculate similarity
        similarities = self._choose_similarity(item_vector, feature_matrix, similarity_metric)
        similarities = pd.DataFrame(similarities, index = feature_matrix.index.tolist())
        similarities.columns = ['similarity_score']
        similarities.sort_values('similarity_score', ascending=False, inplace=True)

        #return top n similar items with similarity scores
        similar_items = similarities.head(n).index.values.tolist()
        scores = similarities.head(n).similarity_score.values.tolist()

        return {"similar_items":similar_items, "score":np.round(scores,5)}
    
    @staticmethod
    def _choose_similarity(item_vector, feature_matrix, similarity_metric):

        if similarity_metric == "cosine":
            similarities =  1-pairwise_distances(X=feature_matrix, Y=item_vector, metric="cosine")
        elif similarity_metric == "euclidean":
            similarities = 1-pairwise_distances(X=feature_matrix, Y=item_vector, metric="euclidean")
        elif similarity_metric == "manhattan":
            similarities = 1 - pairwise_distances(X=feature_matrix, Y=item_vector, metric="manhattan")
        elif similarity_metric == "jaccard":
            similarities = 1 - pairwise_distances(X=feature_matrix, Y=item_vector, metric="hamming")
        return similarities

In [62]:
CF = ContentRecommender()
CF.get_similar_recommendations(seed_item=0, feature_matrix=df, similarity_metric="manhattan", n=10)

{'similar_items': [0, 15, 18, 11, 8, 9, 16, 1, 12, 4],
 'score': array([1.     , 0.75601, 0.7208 , 0.70066, 0.61059, 0.56754, 0.54258,
        0.43809, 0.43722, 0.43545])}